# Training a Convolution Neural Network

In [1]:
import cv2
import matplotlib.pyplot as plt
import sys
import pandas as pd
import os
import glob
import numpy as np
import random
import tqdm
from tqdm import tqdm, tqdm_pandas
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore")

#### Create path variables to dataset

In [2]:
DATADIR = "D:/Projects/diabetic_retinopathy_IDRD/Disease Grading/"
train_images = os.path.join(DATADIR, "Images/Training Set")
train_labels = os.path.join(DATADIR, "Labels/Training_Labels.csv")

test_images = os.path.join(DATADIR, "Images/Testing Set")
test_labels = os.path.join(DATADIR, "Labels/Testing_Labels.csv")

In [3]:
train_image_path = glob.glob(os.path.join(train_images, '*.jpg'))
train_image_path_dict = {os.path.splitext(os.path.basename(x))[0]:x for x in train_image_path}

test_image_path = glob.glob(os.path.join(test_images, '*.jpg'))
test_image_path_dict = {os.path.splitext(os.path.basename(x))[0]:x for x in test_image_path}

#### Read label

In [4]:
df = pd.read_csv(train_labels)
del df['Risk of macular edema']
df['train_image_path'] = df['Image_name'].map(train_image_path_dict.get)
print(df.head())

df2 = pd.read_csv(test_labels)
del df2['Risk of macular edema']
df2['test_image_path'] = df['Image_name'].map(train_image_path_dict.get)
print(df2.head())

  Image_name  Retinopathy_grade  \
0  IDRiD_001                  3   
1  IDRiD_002                  3   
2  IDRiD_003                  2   
3  IDRiD_004                  3   
4  IDRiD_005                  4   

                                    train_image_path  
0  D:/Projects/diabetic_retinopathy_IDRD/Disease ...  
1  D:/Projects/diabetic_retinopathy_IDRD/Disease ...  
2  D:/Projects/diabetic_retinopathy_IDRD/Disease ...  
3  D:/Projects/diabetic_retinopathy_IDRD/Disease ...  
4  D:/Projects/diabetic_retinopathy_IDRD/Disease ...  
  Image_name  Retinopathy_grade  \
0  IDRiD_001                  4   
1  IDRiD_002                  4   
2  IDRiD_003                  4   
3  IDRiD_004                  4   
4  IDRiD_005                  4   

                                     test_image_path  
0  D:/Projects/diabetic_retinopathy_IDRD/Disease ...  
1  D:/Projects/diabetic_retinopathy_IDRD/Disease ...  
2  D:/Projects/diabetic_retinopathy_IDRD/Disease ...  
3  D:/Projects/diabetic_reti

#### Function to create dataset

In [5]:
IMAGE_SIZE = (256, 256)

def create_dataset():
    dataset = []
    for index in df.index:
        image = cv2.resize(cv2.imread(df.train_image_path[index], cv2.IMREAD_GRAYSCALE), IMAGE_SIZE)
        label = df.Retinopathy_grade[index]
        dataset.append([image, label])
    return dataset

def create_test_dataset():
    dataset = []
    for index in df2.index:
        image = cv2.resize(cv2.imread(df2.test_image_path[index], cv2.IMREAD_GRAYSCALE), IMAGE_SIZE)
        label = df2.Retinopathy_grade[index]
        dataset.append([image, label])
    return dataset

def create_sample_dataset(X):
    x = []
    y = []

    for features, label in X:
            x.append(features)
            y.append(label)

    x = np.array(x).reshape(-1, IMAGE_SIZE[0], IMAGE_SIZE[1],1)
    y = np.array(y)

    return x, y

In [6]:
#Create Dataset
image_dataset = create_dataset()

#Shuffle dataset randomnly
random.shuffle(image_dataset)

In [ ]:
x, y = create_sample_dataset(image_dataset)
print(x.shape, y.shape)

In [ ]:
#Create Dataset
test_dataset = create_test_dataset()

#Shuffle dataset randomnly
random.shuffle(test_dataset)

In [ ]:
tx, ty = create_sample_dataset(test_dataset)
print(tx.shape, ty.shape)

#### Create model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

# Compile model
model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

# Fit the model
history = model.fit(x, y, epochs=75, batch_size=10)

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

#### Training Accuracy

In [ ]:
scores = model.evaluate(x, y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#### Saving the model to use

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# Load model

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

#### Evaluate loaded model on test data

In [ ]:
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(tx, ty, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))